In [25]:
# api_key = "eb681b1d49ccf024a0552cd3dba1db8a"

api_key = "cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b"

In [72]:
import requests
import pandas as pd
import json

def get_team_season_players(team_id, season):
    """ Extract json with all players info of a given team and season
        Args: team_id -- int, season (YYYY-YYYY)
    """
    url = f"https://api-football-v1.p.rapidapi.com/v2/players/squad/{team_id}/{season}"
    headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': 'cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b'}
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [20]:
json = get_team_season_players('605', '2019-2020')

In [17]:
def get_players_dict(team_id, season):
    """ Get dict of all players for given team and season where 
         (key, value) -> (player_name, (player_id, p_firstname, p_lastname))
         Args: team_id -- int, season (YYYY-YYYY)
    """
    players_dict = {}
    json = get_team_season_players(team_id, season)['api']['players']
    for i in range(len(json)):
        players_dict[json[i]['player_name']] = (json[i]['player_id'], json[i]['firstname'], json[i]['lastname'])
    return players_dict

In [ ]:
barcelona_id = 529
atletico_madrid_id = 530

In [21]:
dct1 = get_players_dict('530', '2018-2019')
dct1

{'Álex dos Santos': (28, 'Alexandre', 'dos Santos Ferreira'),
 'A. Solano': (40, 'Andrés Felipe', 'Solano Dávila'),
 'D. Godín': (32, 'Diego Roberto', 'Godín Leal'),
 'Vitolo': (46, 'Víctor', 'Machín Pérez'),
 'Juanfran': (41, 'Juan Francisco', 'Torres Belén'),
 'N. Kalinić': (57, 'Nikola', 'Kalinić'),
 'Roberto Olabe': (47033, 'Roberto', 'Olabe del Arco'),
 'Cristian Rodríguez': (51, 'Cristian', 'Rodríguez Pérez'),
 'Mikel Carro': (43, 'Mikel', 'Carro Fandiño'),
 'Álvaro Morata': (59, 'Álvaro Borja', 'Morata Martín'),
 'S. Arias': (30, 'Santiago', 'Arias Naranjo'),
 'Rodri': (44, 'Rodrigo', 'Hernández Cascante'),
 'J. Oblak': (29, 'Jan', 'Oblak'),
 'Toni Moya': (47, 'Antonio', 'Moya Vega'),
 'Joaquín Muñoz': (60, 'Joaquín', 'Muñoz Benavides'),
 'Carlos Isaac': (36, 'Carlos Isaac', 'Muñoz Obejero'),
 'Tachi': (38, 'Alberto', 'Rodríguez Baró'),
 'Borja Garcés': (55, 'Borja', 'Garcés Moreno'),
 'Víctor Mollejo': (58, 'Víctor', 'Mollejo Carpintero'),
 'Gustavo Assunção': (42,
  'Gustavo H

In [22]:
dct2 = get_players_dict('529', '2019-2020')
dct2

{'Dani Morer': (104901, 'Daniel', 'Morer Cabrera'),
 'Piqué': (136, 'Gerard', 'Piqué Bernabéu'),
 'Rafinha': (142, 'Rafael', 'Alcântara do Nascimento'),
 'Nélson Semedo': (130, 'Nélson', 'Cabral Semedo'),
 'K. Saverio': (163436, 'Javier Enrique', 'Delgado Saverio'),
 'Junior Firpo': (1564, 'Héctor Junior', 'Firpo Adames'),
 'L. Reis': (38124, 'Ludovit', 'Reis'),
 'Monchu': (104941, 'Ramón', 'Rodríguez Jiménez'),
 'M. Braithwaite': (2737, 'Martin', 'Christensen Braithwaite'),
 'K. de La Fuente': (161929, 'Konrad', 'de La Fuente'),
 'Jandro Orellana': (162474, 'Alejandro', 'Orellana Gómez'),
 'L. Messi': (154, 'Lionel Andrés', 'Messi Cuccittini'),
 'Sergio Busquets': (144, 'Sergio', 'Busquets Burgos'),
 'Jordi Alba': (128, 'Jordi', 'Alba Ramos'),
 'Riqui Puig': (148, 'Ricard', 'Puig Martí'),
 'Abel Ruiz': (155, 'Abel', 'Ruiz Ortega'),
 'Malcom': (156, 'Malcom Filipe', 'Silva de Oliveira'),
 'R. Manaj': (46725, 'Rey', 'Manaj'),
 'Álex Collado': (146, 'Álex', 'Collado Gutiérrez'),
 'A. Vid

In [86]:
def get_player_stats_df(player_id, team1_id, team2_id, year1, year2):
    """ Replace player_id arg with player_name and use the fns above to get the id
        Args: player_id (int), season (YYYY)
    """
    url1 = f"https://api-football-v1.p.rapidapi.com/v3/players?id={player_id}&season={year1}"
    url2 = f"https://api-football-v1.p.rapidapi.com/v3/players?id={player_id}&season={year2}"
    headers = {
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
            'x-rapidapi-key': 'cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b'}
    response1 = requests.request("GET", url1, headers=headers).json()
    response2 = requests.request("GET", url2, headers=headers).json()
    
    columns_to_remove = ['team.logo', 'league.id', 'league.logo', 'league.flag', 'games.number', 'games.position',
                     ]
    df1 = pd.json_normalize(response1['response'][0]['statistics'])
    df1 = df1[df1['team.id'] == team1_id]
    
    df2 = pd.json_normalize(response2['response'][0]['statistics'])
    df2 = df2[df2['team.id'] == team2_id]
    
    df = pd.concat([df1, df2], ignore_index=True)
    df = df.drop(columns=columns_to_remove)
    return df

In [98]:
df = get_player_stats_df(56, 530, 529, 2018, 2019)
df

,team.id,team.name,league.name,league.country,league.season,games.appearences,games.lineups,games.minutes,games.rating,games.captain,...,fouls.drawn,fouls.committed,cards.yellow,cards.yellowred,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved
0,530,Atletico Madrid,La Liga,Spain,2018,37,37,3204,7.405405,False,...,35.0,18.0,5,0,0,None,None,3.0,0.0,None
1,530,Atletico Madrid,Copa del Rey,Spain,2018,2,1,88,7.100000,False,...,1.0,1.0,1,0,0,None,None,0.0,0.0,None
2,530,Atletico Madrid,UEFA Champions League,World,2018,8,8,720,7.525000,False,...,5.0,7.0,2,0,0,None,None,0.0,0.0,None
3,530,Atletico Madrid,UEFA Super Cup,World,2018,1,1,57,6.200000,False,...,0.0,0.0,0,0,0,None,None,0.0,0.0,None
4,530,Atletico Madrid,International Champions Cup,World,2018,1,0,29,None,False,...,NaN,NaN,0,0,0,None,None,NaN,NaN,None
5,529,Barcelona,La Liga,Spain,2019,35,31,2561,7.018181,False,...,21.0,10.0,2,0,0,None,None,0.0,0.0,None
6,529,Barcelona,Copa del Rey,Spain,2019,3,2,195,7.500000,False,...,0.0,1.0,0,0,0,None,None,0.0,0.0,None
7,529,Barcelona,UEFA Champions League,World,2019,9,7,656,6.733333,False,...,5.0,9.0,2,0,0,None,None,0.0,0.0,None
8,529,Barcelona,Club Friendlies,None,2019,4,4,205,None,False,...,NaN,NaN,0,0,0,None,None,NaN,NaN,None


In [99]:
df.columns.tolist()

['team.id',
 'team.name',
 'league.name',
 'league.country',
 'league.season',
 'games.appearences',
 'games.lineups',
 'games.minutes',
 'games.rating',
 'games.captain',
 'substitutes.in',
 'substitutes.out',
 'substitutes.bench',
 'shots.total',
 'shots.on',
 'goals.total',
 'goals.conceded',
 'goals.assists',
 'goals.saves',
 'passes.total',
 'passes.key',
 'passes.accuracy',
 'tackles.total',
 'tackles.blocks',
 'tackles.interceptions',
 'duels.total',
 'duels.won',
 'dribbles.attempts',
 'dribbles.success',
 'dribbles.past',
 'fouls.drawn',
 'fouls.committed',
 'cards.yellow',
 'cards.yellowred',
 'cards.red',
 'penalty.won',
 'penalty.commited',
 'penalty.scored',
 'penalty.missed',
 'penalty.saved']

In [94]:
url = "https://api-football-v1.p.rapidapi.com/v3/players?id=56&season=2018"
headers = {
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
            'x-rapidapi-key': 'cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b'}
response = requests.request("GET", url, headers=headers).json()

In [95]:
griz_json = response
# griz_json
# print(griz_json.keys())
griz_json['response'][0]['statistics']

[{'team': {'id': 530,
   'name': 'Atletico Madrid',
   'logo': 'https://media-4.api-sports.io/football/teams/530.png'},
  'league': {'id': 140,
   'name': 'La Liga',
   'country': 'Spain',
   'logo': 'https://media-4.api-sports.io/football/leagues/140.png',
   'flag': 'https://media-4.api-sports.io/flags/es.svg',
   'season': 2018},
  'games': {'appearences': 37,
   'lineups': 37,
   'minutes': 3204,
   'number': None,
   'position': 'Attacker',
   'rating': '7.405405',
   'captain': False},
  'substitutes': {'in': 0, 'out': 6, 'bench': 0},
  'shots': {'total': 113, 'on': 46},
  'goals': {'total': 15, 'conceded': None, 'assists': 9, 'saves': None},
  'passes': {'total': 1087, 'key': 74, 'accuracy': 77},
  'tackles': {'total': 19, 'blocks': 1, 'interceptions': 12},
  'duels': {'total': 292, 'won': 127},
  'dribbles': {'attempts': 44, 'success': 20, 'past': None},
  'fouls': {'drawn': 35, 'committed': 18},
  'cards': {'yellow': 5, 'yellowred': 0, 'red': 0},
  'penalty': {'won': None,
   

In [96]:
import pandas as pd
import json

df = pd.json_normalize(griz_json['response'][0]['statistics'])
df.columns.tolist()

# List of column names to remove
columns_to_remove = ['team.logo', 'league.id', 'league.logo', 'league.flag', 'games.number', 'games.position',
                     ]

# Drop the specified columns
df = df.drop(columns=columns_to_remove)
df

,team.id,team.name,league.name,league.country,league.season,games.appearences,games.lineups,games.minutes,games.rating,games.captain,...,fouls.drawn,fouls.committed,cards.yellow,cards.yellowred,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved
0,530,Atletico Madrid,La Liga,Spain,2018,37,37,3204,7.405405,False,...,35.0,18.0,5,0,0,None,None,3.0,0.0,None
1,530,Atletico Madrid,Copa del Rey,Spain,2018,2,1,88,7.100000,False,...,1.0,1.0,1,0,0,None,None,0.0,0.0,None
2,530,Atletico Madrid,UEFA Champions League,World,2018,8,8,720,7.525000,False,...,5.0,7.0,2,0,0,None,None,0.0,0.0,None
3,530,Atletico Madrid,UEFA Super Cup,World,2018,1,1,57,6.200000,False,...,0.0,0.0,0,0,0,None,None,0.0,0.0,None
4,530,Atletico Madrid,International Champions Cup,World,2018,1,0,29,None,False,...,NaN,NaN,0,0,0,None,None,NaN,NaN,None
5,2,France,UEFA Nations League,World,2018,4,4,341,None,False,...,NaN,NaN,1,0,0,None,None,NaN,NaN,None
6,2,France,World Cup - Qualification Europe,World,2018,10,10,855,7.400000,False,...,4.0,3.0,1,0,0,None,None,0.0,0.0,None
7,2,France,World Cup,World,2018,7,7,571,7.400000,False,...,11.0,6.0,0,0,0,None,None,3.0,0.0,None
8,2,France,Friendlies,World,2018,7,5,437,None,False,...,NaN,NaN,1,0,0,None,None,NaN,NaN,None


In [97]:
df2 = df[df['team.id'] == 530]
df2

,team.id,team.name,league.name,league.country,league.season,games.appearences,games.lineups,games.minutes,games.rating,games.captain,...,fouls.drawn,fouls.committed,cards.yellow,cards.yellowred,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved
0,530,Atletico Madrid,La Liga,Spain,2018,37,37,3204,7.405405,False,...,35.0,18.0,5,0,0,None,None,3.0,0.0,None
1,530,Atletico Madrid,Copa del Rey,Spain,2018,2,1,88,7.100000,False,...,1.0,1.0,1,0,0,None,None,0.0,0.0,None
2,530,Atletico Madrid,UEFA Champions League,World,2018,8,8,720,7.525000,False,...,5.0,7.0,2,0,0,None,None,0.0,0.0,None
3,530,Atletico Madrid,UEFA Super Cup,World,2018,1,1,57,6.200000,False,...,0.0,0.0,0,0,0,None,None,0.0,0.0,None
4,530,Atletico Madrid,International Champions Cup,World,2018,1,0,29,None,False,...,NaN,NaN,0,0,0,None,None,NaN,NaN,None


In [51]:
url = "https://api-football-v1.p.rapidapi.com/v3/transfers?player=56"

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': 'cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b'}
transfer = requests.request("GET", url, headers=headers).json()

In [52]:
transfer

{'get': 'transfers',
 'parameters': {'player': '56'},
 'errors': [],
 'results': 1,
 'paging': {'current': 1, 'total': 1},
 'response': [{'player': {'id': 56, 'name': 'A. Griezmann'},
   'update': '2023-11-02T12:22:00+00:00',
   'transfers': [{'date': '2023-07-01',
     'type': '€ 20M',
     'teams': {'in': {'id': 530,
       'name': 'Atletico Madrid',
       'logo': 'https://media-4.api-sports.io/football/teams/530.png'},
      'out': {'id': 529,
       'name': 'Barcelona',
       'logo': 'https://media-4.api-sports.io/football/teams/529.png'}}},
    {'date': '2023-07-01',
     'type': '€ 20M',
     'teams': {'in': {'id': 530,
       'name': 'Atletico Madrid',
       'logo': 'https://media-4.api-sports.io/football/teams/530.png'},
      'out': {'id': 529,
       'name': 'Barcelona',
       'logo': 'https://media-4.api-sports.io/football/teams/529.png'}}},
    {'date': '2021-08-31',
     'type': 'Loan',
     'teams': {'in': {'id': 530,
       'name': 'Atletico Madrid',
       'logo': '

Some bad transfers (from https://www.90min.com/posts/transfers-worst-signings-football-history-hazard-coutinho-dembele):

- Antoine Griezmann (Atletico Madrid to Barcelona, 2019) -- Approx transfer fee: €120m
- Eden Hazard (Chelsea to Real Madrid, 2019) -- Approx transfer fee: €100m (rising to €146m with add-ons)
- Philippe Coutinho (Liverpool to Barcelona, 2018) -- Approx transfer fee: £105m (rising to £142m with add-ons)
- Romelu Lukaku (Inter to Chelsea, 2021) -- Approx transfer fee: £97.5m
- Carlos Tevez (Boca Juniors to Shanghai Shenhua, 2017) -- Approx transfer fee: €16m (transfer fee is disputed)
- Ousmane Dembele (Borussia Dortmund to Barcelona, 2017) -- Approx transfer fee: €105m (rising to €145m with add-ons)
- ~~Kepa Arrizabalaga (Athletic Club to Chelsea, 2018) -- Approx transfer fee: £72m~~
- Alexis Sanchez (Arsenal to Manchester United, 2018) -- Approx transfer fee: Swap with Henrikh Mkhitaryan
- ~~Danny Drinkwater (Leicester to Chelsea, 2017) -- Approx transfer fee: £35m~~
- Paul Pogba (Juventus to Man Utd, 2016) -- Approx transfer fee: £89m
- Andriy Shevchenko (AC Milan to Chelsea, 2006) -- Approx transfer fee: £30m
- Zlatan Ibrahimovic (Inter to Barcelona, 2009) -- Approx transfer fee: £59m + Samuel Eto'o
- ~~Gaizka Mendieta (Valencia to Lazio, 2001) -- Approx transfer fee: €48m~~
- Andy Carroll (Newcastle United to Liverpool, 2011) -- Approx transfer fee: £35m
- Luka Jovic (Eintracht Frankfurt to Real Madrid, 2019) -- Approx transfer fee: €60m

In [137]:
# Code to compare player's dfs from last season before transfer and first season after transfer
# Do this for all bad transfers

player_ids = {'griezmann': 140416, 'hazard': 101390, 'coutinho': 130372, 'lukaku': 139437, 'tevez': 72490,
              'dembele': 229744, 'arrizabalaga': 169532, 'sanchez': 92055, 'drinkwater': 135376, 'pogba': 138860,
              'shevchenko': 11884, 'ibrahimovic': 11001, 'carroll': 91478, 'jovic': 221332}

team_ids = {'atletico_madrid': 1068, 'barcelona': 83, 'chelsea': 363, 'real_madrid': 86, 'liverpool': 364, 
            'internazionale': 110, 'boca-juniors' :5, 'shanghai-shenhua': 977, 'borussia-dortmund': 124, 
            'athletic-club': 93, 'arsenal': 359, 'manchester-united': 360, 'leicester-city': 375, 'juventus': 111,
            'ac-milan': 103, 'valencia': 94, 'lazio': 112, 'newcastle-united': 361, 'eintracht-frankfurt': 125}

transfers = {
    'griezmann': (140416, 1068, 83, '19'),
    'hazard': (101390, 363, 86, '19'),
    'coutinho': (130372, 364, 83, '18'),
    'lukaku': (139437, 110, 363, '21'),
    'tevez': (72490, 5, 977, '17'),
    'dembele': (229744, 124, 83, '17'),
    'sanchez': (92055, 359, 360, '18'),
    'pogba': (138860, 111, 360, '16'),
    'shevchenko': (11884, 103, 363, '06'),
    'ibrahimovic': (11001, 110, 83, '09'),
    'carroll': (91478, 361, 364, '11'),
    'jovic': (221332, 125, 86, '19')
}

cols = ['Season', 'Team', 'Strt', 'FC', 'FA', 'YC', 'RC', 'G', 'A', 'SH', 'ST', 'OF']

In [116]:
import pandas as pd
from IPython.display import display

def compare_transfer_seasons(player_id, team1_id, team2_id, transfer_year):
    url1 = f"https://www.espn.com/soccer/player/stats/_/id/{player_id}/team/{team1_id}"
    url2 = f"https://www.espn.com/soccer/player/stats/_/id/{player_id}/team/{team2_id}"
    before_tables = pd.read_html(url1)
    after_tables = pd.read_html(url2)
    
    before_df, after_df = pd.DataFrame(), pd.DataFrame()
    
    for table1, table2 in zip(before_tables, after_tables):
        before_df = pd.concat([before_df, table1], ignore_index=True, axis=1)
        after_df = pd.concat([after_df, table2], ignore_index=True, axis=1)
        
#     for table in before_tables:
#         before_df = pd.concat([before_df, table], ignore_index=True, axis=1)
    
#     for table in after_tables:
#         after_df = pd.concat([after_df, table], ignore_index=True, axis=1)
    
    condition1 = before_df.iloc[:, 0].str.contains(transfer_year, na=False)
    before_df = before_df[condition1]
    
    condition2 = after_df.iloc[:, 0].str.contains(transfer_year, na=False)
    after_df = after_df[condition2]
    
    transfer_df = pd.concat([before_df, after_df], ignore_index=True, axis=0)
    transfer_df.columns = cols
    
    return transfer_df

In [138]:
dfs_list = []
for player in transfers:
    player_id, team1_id, team2_id, year = transfers[player]
    df = compare_transfer_seasons(player_id, team1_id, team2_id, year)
    df = df.rename_axis(player)
    dfs_list.append(df)

In [140]:
for df in dfs_list:
    display(df)

,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
griezmann,,,,,,,,,,,,
0,2018-19 Spanish LaLiga,ATM,37,18,35,5,0,15,9,113,46,12
1,2019-20 LaLiga,BAR,31,10,21,2,0,9,4,55,23,16


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
hazard,,,,,,,,,,,,
0,2018-19 English Premier League,CHE,32,12,104,2,0,16,15,93,43,11
1,2019-20 LaLiga,RMA,14,12,50,0,0,1,3,19,13,2


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
coutinho,,,,,,,,,,,,
0,2017-18 English Premier League,LIV,13,5,21,0,0,7,6,54,17,4
1,2018-19 Spanish LaLiga,BAR,22,19,45,2,0,5,2,68,32,8
2,2017-18 Spanish LaLiga,BAR,16,12,32,1,0,8,5,37,20,8


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
lukaku,,,,,,,,,,,,
0,2020-21 Italian Serie A,INT,32,31,48,4,0,24,11,95,48,14
1,2021-22 English Premier League,CHE,16,8,12,1,0,8,0,40,13,3


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
tevez,,,,,,,,,,,,
0,2017-18 Argentine Superliga,CABJ,10,4,20,0,0,3,2,28,10,3
1,2016/17 Argentina Primera Division,CABJ,11,10,25,1,1,5,7,32,14,3
2,2017 Chinese Super League,SHE,14,18,39,1,0,4,5,53,15,6


...

,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
pogba,,,,,,,,,,,,
0,2015-16 Italian Serie A,JUV,33,88,57,10,0,8,12,124,39,13
1,2016-17 English Premier League,MAN,29,62,71,7,0,5,4,92,32,3


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
shevchenko,,,,,,,,,,,,
0,2005-06 Italian Serie A,MIL,24,0,0,1,0,19,0,0,0,0
1,2006-07 Barclays Premier League,CHE,22,11,27,1,0,4,6,54,36,18


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
ibrahimovic,,,,,,,,,,,,
0,2008-09 Italian Serie A,INT,35,81,73,8,0,25,7,158,68,75
1,2009-10 Spanish Primera División,BAR,23,59,34,6,1,16,7,100,44,52


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
carroll,,,,,,,,,,,,
0,2010-11 Barclays Premier League,NEW,18,47,31,4,0,11,3,75,26,5
1,2011-12 Barclays Premier League,LIV,11,14,32,3,0,1,1,55,15,7
2,2010-11 Barclays Premier League,LIV,5,6,6,1,0,2,0,16,7,4


,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
jovic,,,,,,,,,,,,
0,2018-19 German Bundesliga,SGE,25,29,43,1,0,17,5,97,43,17
1,2019-20 LaLiga,RMA,4,5,1,2,0,2,1,14,3,4


In [65]:
url = "https://www.espn.com/soccer/player/stats/_/id/140416/team/1068"
tables = pd.read_html(url)

atm_df = pd.DataFrame()
for table in tables:
#     table = pd.DataFrame(table)
    atm_df = pd.concat([atm_df, table], ignore_index=True, axis=1)
atm_df.columns = cols
atm_df

,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
0,2023-24 LALIGA,ATM,11,5,8,3,0,7,0,23,10,2
1,2022-23 LaLiga,ATM,31,21,41,2,0,15,16,109,42,4
2,2021-22 LaLiga,ATM,21,18,27,3,0,3,4,43,13,5
3,2018-19 Spanish LaLiga,ATM,37,18,35,5,0,15,9,113,46,12
4,2017-18 Spanish LaLiga,ATM,30,24,44,6,1,19,9,72,40,26
5,2016-17 Spanish LaLiga,ATM,36,28,33,3,0,16,8,88,41,27
6,2015-16 Spanish LaLiga,ATM,36,31,37,5,0,22,5,92,43,17
7,2014-15 Spanish Primera División,ATM,29,39,31,6,0,22,1,78,38,26


In [59]:
condition = atm_df.iloc[:, 0].str.contains('19', na=False)
atm_df = atm_df[condition]
atm_df

,Season,Team,Strt,FC,FA,YC,RC,G,A,SH,ST,OF
3,2018-19 Spanish LaLiga,ATM,37,18,35,5,0,15,9,113,46,12
